In [58]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import patsy
import imp
import os


plt.style.use('fivethirtyeight')

from sklearn import grid_search, datasets
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import cross_val_score, cross_val_predict, train_test_split, StratifiedKFold
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression as lm, LogisticRegression as lr, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from ipywidgets import *
from IPython.display import display
from collections import defaultdict

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

business = pd.read_csv('/Users/nichollejames/Desktop/DSI-SF-2-oboechick/projects/project-05/yelp_arizona_data/businesses_small_parsed.csv')
# checkins = pd.read_csv('/Users/nichollejames/Desktop/DSI-SF-2-oboechick/projects/project-05/yelp_arizona_data/checkins_small_parsed.csv')
# review = pd.read_csv('/Users/nichollejames/Desktop/DSI-SF-2-oboechick/projects/project-05/yelp_arizona_data/reviews_small_nlp_parsed.csv')
# tips = pd.read_csv('/Users/nichollejames/Desktop/DSI-SF-2-oboechick/projects/project-05/yelp_arizona_data/tips_small_nlp_parsed.csv')
# users = pd.read_csv('/Users/nichollejames/Desktop/DSI-SF-2-oboechick/projects/project-05/yelp_arizona_data/users_small_parsed.csv')


In [59]:
# All .xls files from C_comp_COG_MAR31

COG_All_items = pd.read_excel("/Users/nichollejames/Desktop/capstone_project/C_comp_COG_MAR31/COMP_CBACOG12_MAR31.xlsx", 
                                  sheetname='All_items', skiprows = 1)

In [60]:
COG_All_items.head(2)

,Unnamed: 0,Unnamed: 1,"""r""",0,1,2,3,4,5,"""Missing/Invalid""",...,3_se,4_se,5_se,Missing/Invalid_se,PCT_corr,PCT_corrBoys,PCT_corrGirls,PCT_corr_SE,PCT_corrBoys_SE,PCT_corrGirls_SE
0,AUS,CM002Q01,1.414989,67.934951,16.285406,0.721712,0.0,0.0,0.0,15.057931,...,0.0,0.0,0.0,0.750994,8.864415,10.590911,7.039116,0.540651,0.805428,0.816535
1,AUT,CM002Q01,3.067767,72.548832,14.329332,0.648698,0.0,0.0,0.0,12.473138,...,0.0,0.0,0.0,1.964565,7.813364,11.497438,3.964885,0.997966,1.687128,1.037837


In [61]:
COG_All_items = COG_All_items.rename(columns={'Unnamed: 0':'cnt',
                                                     'Unnamed: 1': 'ItemID',
                                                      '"r"': 'percent_by_category_r',
                                                      0: 'percent_by_category_0',
                                                      1: 'percent_by_category_1',
                                                      2: 'percent_by_category_2',
                                                      3: 'percent_by_category_3',
                                                      4: 'percent_by_category_4',
                                                      5: 'percent_by_category_5',
                                                      '"Missing/Invalid"': 'percent_by_category_missing_invalid',
                                                      'r_se': 'standard_error_of_the_percent_by_category_r_se',
                                                      '0_se': 'standard_error_of_the_percent_by_category_0_se',
                                                      '1_se': 'standard_error_of_the_percent_by_category_1_se',
                                                      '2_se': 'standard_error_of_the_percent_by_category_2_se',
                                                      '3_se': 'standard_error_of_the_percent_by_category_3_se',
                                                      '4_se': 'standard_error_of_the_percent_by_category_4_se',
                                                      '5_se': 'standard_error_of_the_percent_by_category_5_se',
                                                      'Missing/Invalid_se': 'standard_error_of_the_percent_by_category_missing_invalid_se'
                                                     })

In [62]:
COG_All_items.head(2)

,cnt,ItemID,percent_by_category_r,percent_by_category_0,percent_by_category_1,percent_by_category_2,percent_by_category_3,percent_by_category_4,percent_by_category_5,percent_by_category_missing_invalid,...,standard_error_of_the_percent_by_category_3_se,standard_error_of_the_percent_by_category_4_se,standard_error_of_the_percent_by_category_5_se,standard_error_of_the_percent_by_category_missing_invalid_se,PCT_corr,PCT_corrBoys,PCT_corrGirls,PCT_corr_SE,PCT_corrBoys_SE,PCT_corrGirls_SE
0,AUS,CM002Q01,1.414989,67.934951,16.285406,0.721712,0.0,0.0,0.0,15.057931,...,0.0,0.0,0.0,0.750994,8.864415,10.590911,7.039116,0.540651,0.805428,0.816535
1,AUT,CM002Q01,3.067767,72.548832,14.329332,0.648698,0.0,0.0,0.0,12.473138,...,0.0,0.0,0.0,1.964565,7.813364,11.497438,3.964885,0.997966,1.687128,1.037837


In [63]:
COG_Double_digit_items = pd.read_excel("/Users/nichollejames/Desktop/capstone_project/C_comp_COG_MAR31/COMP_CBACOG12_MAR31.xlsx", 
                                  sheetname='Double_digit_items', skiprows = 1)

In [64]:
COG_Double_digit_items = COG_Double_digit_items.rename(columns={'Unnamed: 0':'cnt',
                                                     'Unnamed: 1': 'ItemID',
                                                      '"r"': 'percent_by_category_r',
                                                      '"00"': 'percent_by_category_00',
                                                      '"11"': 'percent_by_category_11',
                                                      '"12"': 'percent_by_category_12',
                                                      '"13"': 'percent_by_category_13',
                                                      '"14"': 'percent_by_category_14',
                                                      '"21"': 'percent_by_category_21',
                                                      '"Missing/Invalid"': 'percent_by_category_missing_invalid',
                                                      '"r"_se': 'standard_error_of_the_percent_by_category_r_se',
                                                      '"00"_se': 'standard_error_of_the_percent_by_category_00_se',
                                                      '"11"_se': 'standard_error_of_the_percent_by_category_11_se',
                                                      '"12"_se': 'standard_error_of_the_percent_by_category_12_se',
                                                      '"13"_se': 'standard_error_of_the_percent_by_category_13_se',
                                                      '"14"_se': 'standard_error_of_the_percent_by_category_14_se',
                                                      '"21"_se': 'standard_error_of_the_percent_by_category_21_se',
                                                      'Missing/Invalid_se': 'standard_error_of_the_percent_by_category_missing_invalid_se'
                                                     })

In [65]:
COG_Double_digit_items.head(1)

,cnt,ItemID,percent_by_category_r,percent_by_category_00,percent_by_category_11,percent_by_category_12,percent_by_category_13,percent_by_category_14,percent_by_category_21,percent_by_category_missing_invalid,...,standard_error_of_the_percent_by_category_13_se,standard_error_of_the_percent_by_category_14_se,standard_error_of_the_percent_by_category_21_se,standard_error_of_the_percent_by_category_missing_invalid_se,PCT_corr,PCT_corrBoys,PCT_corrGirls,PCT_corr_SE,PCT_corrBoys_SE,PCT_corrGirls_SE
0,AUS,CM015Q02D,0.20971,58.842914,2.469419,0.095028,0.225882,1.030397,8.658412,28.677949,...,0.131829,0.259646,0.655093,1.254858,10.568775,12.046066,8.994895,0.675305,0.935628,0.988621


In [84]:
# All .xls files from C_comp_EC_MAR31

EC = defaultdict()
sheets = ['Perc', 'Mean_C_PROB', 'Mean_C_MATH', 'Mean_C_READ']
keys = []

base_dir = "/Users/nichollejames/Desktop/capstone_project/C_comp_EC_MAR31/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        keys.append(key)
        
        for sheet in sheets:
            
            if key not in EC:
                EC[key] = {}
            
            EC[key].update({
                sheet:  pd.read_excel(base_dir+f, sheetname=sheet, skiprows = 1)       
            })

# EC[x][y] change all of the column names

for x in EC:
    for y in sheets:
        EC[x][y] = EC[x][y].rename(columns={' ': 'cnt',
                                            'N': 'total_number_of_resp',
                                            'N/A': 'pct_not_valid',
                                            'N.1': 'no_of_valid_resp',
                                            'No, never': 'resp_no_never_pct',
                                            'No, never.1': 'resp_no_never_se',
                                            'No, never  .1': 'resp_no_never_se',
                                            'No, never  ': 'res_no_never_mean',
                                            'Yes, once' : 'resp_yes_once_pct',
                                            'Yes, once  .1': 'resp_yes_once_se',
                                            'Yes, once  ': 'resp_yes_once_mean',
                                            'Yes, twice or more': 'resp_yes_twice_or_more_pct',
                                            'Yes, twice or more  .1': 'resp_yes_twice_or_more_se',
                                            'Yes, twice or more  ': 'resp_yes_twice_or_more_mean',
                                            'Mostly <heritage language> ': 'resp_mostly_heritage_language_pct',
                                            'Mostly <heritage language> .1': 'resp_mostly_heritage_languagee_se',
                                            'About equally <heritage language> and <test language>': 'resp_about_equally_heritage_language_and_test_language_pct',
                                            'About equally <heritage language> and <test language>.1': 'resp_about_equally_heritage_language_and_test_language_se',
                                            'Mostly <test language>': 'resp_mostly_test_language_pct',
                                            'Mostly <test language>.1': 'resp_mostly_test_language_se',
                                            'Not applicable': 'resp_not_applicable_pct',
                                            'Not applicable.1': 'resp_not_applicable_se',
                                            'Tick': 'resp_tick_pct',
                                            'Tick.1': 'resp_tick_se',
                                            'No tick': 'resp_no_tick_pct',
                                            'No tick.1': 'resp_no_tick_se',
                                            'Strongly agree': 'resp_strongly_agree_pct',
                                            'Strongly agree.1': 'resp_strongly_agree_se',
                                            'Agree': 'resp_agree_pct',
                                            'Agree.1': 'resp_agree_se',
                                            'Disagree': 'resp_disagree_pct',
                                            'Disagree.1': 'resp_disagree_se',
                                            'Strongly disagree': 'resp_strongly_disagree_pct',
                                            'Strongly disagree.1': 'resp_strongly_disagree_se',
                                            'No': 'resp_no_pct',
                                            'No.1': 'resp_no_se',
                                            'Yes': 'resp_yes_pct',
                                            'Yes.1': 'resp_yes_se',
                                            'Missing': 'pct__resp_missing',
                                            'Missing.1': 'resp_missing_se',
                     })
        
        
# Run this only once or the column names will have the keys repeated.

lvl1keys = EC.keys()

for l1key in lvl1keys:
    lvl2keys = EC[l1key].keys()
#     print lvl2keys
    for l2key in lvl2keys:
        current_columns = EC[l1key][l2key].columns
        EC[l1key][l2key].columns = [l1key+'_'+l2key+'_'+x if x!='cnt' else x for x in current_columns]
        

# # Merge all the dataframes together

# df_EC = pd.DataFrame(EC['EC01Q01']['Perc'])

# for filename, worksheets in EC.items():
#     print "Merging: ", filename

#     for worksheet, sheet_df in worksheets.items():
#         # print worksheet, sheet_df.columns
#         df_EC = pd.merge(df_EC, sheet_df, how="inner", on="cnt")


# # Delete the first row since it was inserted into the column headers

# df_EC = df_EC[df_EC.EC01Q01_Perc_total_number_of_resp_x != 'ALL']

df = df[df.EC01Q01_Perc_total_number_of_resp != 'ALL']

In [85]:
df.head()

,cnt,EC01Q01_Perc_total_number_of_resp,EC01Q01_Perc_pct_not_valid,EC01Q01_Perc_no_of_valid_resp,EC01Q01_Perc_res_no_never_mean,EC01Q01_Perc_resp_no_never_se,EC01Q01_Perc_resp_yes_once_mean,EC01Q01_Perc_resp_yes_once_se,EC01Q01_Perc_resp_yes_twice_or_more_mean,EC01Q01_Perc_resp_yes_twice_or_more_se,EC01Q01_Perc_pct__resp_missing,EC01Q01_Perc_resp_missing_se
1,Australia,250711,1.17439,247767,81.2257,(.51),11.7966,(.40),2.84758,(.14),4.13008,(.23)
2,Austria,82225,100,0,,,,,,,,
3,Belgium,117889,3.90119,113290,88.9528,(.45),6.5268,(.31),1.48633,(.16),3.0341,(.32)
4,Canada,347987,1.38883,343154,88.088,(.35),8.05146,(.27),1.7822,(.14),2.07839,(.21)
5,Chile,229159,100,0,,,,,,,,


In [94]:
## Define y
y = df['cnt']

## Define X (exclude columns 'male', 'incsq' ,'agesq') on X
columns_ = df.columns.tolist()
exclude_cols = ['EC01Q01_Perc_pct_not_valid']
X = df[[i for i in columns_ if i not in exclude_cols]]

## Print shapes of y and X
print y.shape, X.shape

(46,) (46, 11)


In [10]:
# All .xls files from C_comp_ICT_MAR31

ICT = defaultdict()
sheets = ['Perc', 'Mean_C_PROB', 'Mean_C_MATH', 'Mean_C_READ']

base_dir = "/Users/nichollejames/Desktop/capstone_project/C_comp_ICT_MAR31/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        
        for sheet in sheets:
            
            if key not in ICT:
                ICT[key] = {}
            
            ICT[key].update({
                sheet:  pd.read_excel(base_dir+f, sheetname=sheet, skiprows = 1)       
            })


# ICT[x][y] change all of the column names

for x in ICT:
    for y in sheets:
        ICT[x][y] = ICT[x][y].rename(columns={' ': 'cnt',
                                            'N': 'total_number_of_resp',
                                            'N/A': 'pct_not_valid',
                                            'N.1': 'no_of_valid_resp',
                                            'No time': 'resp_no_time_pct',
                                            'No time.1': 'resp_no_time_se',
                                            'No time  ': 'resp_no_time_mean',
                                            'Yes, students did this': 'resp_yes_students_did_this_pct',
                                            'Yes, students did this.1': 'resp_yes_students did_this_time_se',
                                            'Yes, but only the teacher demonstrated this': 'resp_yes_but_only_the_teacher_demonstrated_this_pct',
                                            'Yes, but only the teacher demonstrated this.1': 'resp_yes_but_only_the_teacher_demonstrated_this_se',
                                            '1-30 minutes': 'resp_1_30_minutes_pct',
                                            '1-30 minutes.1': 'resp_1_30_minutes_se',
                                            '31-60 minutes': 'resp_31_60_minutes_pct',
                                            '31-60 minutes.1': 'resp_31_60_minute_se',
                                            'Between 1 and 2 hours': 'resp_between_1_and_2_hours_pct',
                                            'Between 1 and 2 hours.1': 'resp_between_1_and_2_hours_se',
                                            'Between 2 and 4 hours': 'resp_between_2_and_4_hours_pct',
                                            'Between 2 and 4 hours.1': 'resp_between_2_and_4_hours_se',
                                            'Between 4 hours and 6 hours': 'resp_between_4_hours_and_6_hours_pct',
                                            'Between 4 hours and 6 hours.1': 'resp_between_4_hours_and_6_hours_se',
                                            'More than 6 hours': 'resp_more_than_6_hours_pct',
                                            'More than 6 hours.1': 'resp_more_than_6_hours_se',
                                            'Never or hardly ever': 'resp_never_or_hardly_ever_pct',
                                            'Never or hardly ever.1': 'resp_never_or_hardly_ever_se',
                                            'Once or twice a month' : 'resp_once_or_twice_a_month_pct',
                                            'Once or twice a month.1': 'resp_once_or_twice_a_month_se',
                                            'Once or twice a week' : 'resp_once_or_twice_a_week_pct',
                                            'Once or twice a week.1': 'resp_once_or_twice_a_week_se',
                                            'Almost every day': 'resp_almost_every_day_pct',
                                            'Almost every day.1': 'resp_almost_every_day_se',
                                            'Yes, and I use it': 'resp_yes_and_i_use_it_pct',
                                            'Yes, and I use it.1': 'resp_yes_and_i_use_it_se',
                                            'Yes, but I don?t use it': 'resp_yes_but_i_dont_use_it_pct',
                                            'Yes, but I don?t use it.1': 'resp_yes_but_i_dont_use_it_se',
                                            'No': 'resp_no_pct',
                                            'No.1': 'resp_no_se',
                                            'Strongly agree': 'resp_strongly_agree_pct',
                                            'Strongly agree.1': 'resp_strongly_agree_se',
                                            'Agree': 'resp_agree_pct',
                                            'Agree.1': 'resp_agree_se',
                                            'Disagree': 'resp_disagree_pct',
                                            'Disagree.1': 'resp_disagree_se',
                                            'Strongly disagree': 'resp_strongly_disagree_pct',
                                            'Strongly disagree.1': 'resp_strongly_disagree_se',
                                            'Every day': 'resp_every_day_pct',
                                            'Every day.1': 'resp_every_day_se',
                                            '6 years old or younger': 'resp_6_years_old_or_younger_pct',
                                            '6 years old or younger.1': 'resp_6_years_old_or_younger_se',
                                            '7-9 years old': 'resp_7_9_years_old_pct',
                                            '7-9 years old.1': 'resp_7_9_years_old_se',
                                            '10-12 years old': 'resp_10_12_years_old_pct',
                                            '10-12 years old.1': 'resp_10_12_years_old_se',
                                            '13 years old or older': 'resp_13_years_old_or_older_pct',
                                            '13 years old or older.1': 'resp_13_years_old_or_older_se',
                                            '13 years old  or older': 'resp_13_years_old_or_older_pct',
                                            '13 years old  or older.1': 'resp_13_years_old_or_older_se',
                                            'Never': 'resp_never_pct',
                                            'Never.1': 'resp_never_se',
                                            'Missing': 'pct__resp_missing',
                                            'Missing.1': 'resp_missing_se'
                     })
        

# Run this only once or the column names will have the keys repeated.

lvl1keys = ICT.keys()

for l1key in lvl1keys:
    lvl2keys = ICT[l1key].keys()
#     print lvl2keys
    for l2key in lvl2keys:
        current_columns = ICT[l1key][l2key].columns
        ICT[l1key][l2key].columns = [l1key+'_'+l2key+'_'+x if x!='cnt' else x for x in current_columns]
        

df_ICT = pd.DataFrame(ICT['IC01Q01']['Perc'])

for filename, worksheets in ICT.items():
    print "Merging: ", filename
    
    for worksheet, sheet_df in worksheets.items():
        # print worksheet, sheet_df.columns
        df_ICT = pd.merge(df_ICT, sheet_df, how="inner", on="cnt")

Merging:  IC22Q08
Merging:  IC01Q09
Merging:  IC01Q08
Merging:  IC08Q08
Merging:  IC08Q09
Merging:  IC01Q05
Merging:  IC01Q04
Merging:  IC01Q07
Merging:  IC04Q01
Merging:  IC01Q01
Merging:  IC08Q03
Merging:  IC01Q03
Merging:  IC01Q02
Merging:  IC05Q01
Merging:  IC22Q01
Merging:  IC10Q09
Merging:  IC06Q01
Merging:  IC10Q07
Merging:  IC10Q06
Merging:  IC10Q05
Merging:  IC10Q04
Merging:  IC10Q03
Merging:  IC10Q02
Merging:  IC10Q01
Merging:  IC09Q04
Merging:  IC11Q02
Merging:  IC08Q04
Merging:  IC10Q08
Merging:  IC09Q05
Merging:  IC08Q06
Merging:  IC09Q07
Merging:  IC09Q06
Merging:  IC09Q01
Merging:  IC11Q03
Merging:  IC09Q03
Merging:  IC08Q07
Merging:  IC11Q04
Merging:  IC07Q01
Merging:  IC11Q05
Merging:  IC08Q11
Merging:  IC01Q06
Merging:  IC11Q06
Merging:  IC08Q02
Merging:  IC01Q10
Merging:  IC01Q11
Merging:  IC08Q05
Merging:  IC22Q07
Merging:  IC22Q04
Merging:  IC08Q01
Merging:  IC11Q07
Merging:  IC09Q02
Merging:  IC03Q01
Merging:  IC22Q06
Merging:  IC02Q01
Merging:  IC02Q02
Merging:  

In [11]:
df_ICT.T.tail(100)

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
IC02Q07_Mean_C_PROB_resp_no_pct,Mean,529.766,512.902,523.02,,453.744,518.153,506.52,523.366,529.178,...,,546.61,544.556,,,495.485,485.19,568.272,,419.885
IC02Q07_Mean_C_PROB_resp_no_se,(SE),(1.98),(3.48),(2.43),,(3.73),(2.90),(2.93),(2.43),(2.14),...,,(3.82),(1.15),,,(3.43),(3.09),(1.35),,(3.36)
IC02Q07_Mean_C_PROB_pct__resp_missing,Mean,461.221,433.415,468.952,,400.925,442.973,461.633,475.407,447.585,...,,470.334,498.868,,,442.018,401.998,531.267,,360.024
IC02Q07_Mean_C_PROB_resp_missing_se,(SE),(6.46),(14.62),(6.81),,(8.70),(10.70),(5.54),(10.17),(9.66),...,,(18.66),(14.88),,,(7.93),(7.67),(12.78),,(5.38)
IC11Q01_Perc_total_number_of_resp,ALL,250711,82225,117889,347987,229159,82250.1,65642,11626.5,60047.2,...,9650.26,70636.3,5366,432080,7714.2,1.17453e+06,67934.2,51088.2,40612.4,39770.9
IC11Q01_Perc_pct_not_valid,%,1.17439,2.05505,3.90119,100,1.08238,0.2604,2.79744,0.696724,1.70522,...,100,4.21986,0.191744,100,100,0.344736,1.01876,0.329601,100,0.831862
IC11Q01_Perc_no_of_valid_resp,Valid,247767,80535.3,113290,0,226678,82035.9,63805.7,11545.5,59023.3,...,0,67655.5,5355.71,0,0,1.17048e+06,67242.1,50919.8,0,39440
IC11Q01_Perc_resp_yes_students_did_this_pct,%,19.582,14.4015,11.4726,,13.3647,10.1132,32.1618,16.5955,8.94628,...,,6.02178,13.9363,,,28.1079,15.1923,22.8339,,22.9845
IC11Q01_Perc_resp_yes_students did_this_time_se,(SE),(.50),(.98),(.74),,(.73),(.73),(1.31),(.74),(.62),...,,(.56),(.47),,,(.90),(.62),(.56),,(.98)
"IC11Q01_Perc_Yes, but only the teacher demonstrated this",%,17.1651,10.6979,12.7581,,10.2928,9.00657,15.84,16.383,9.26122,...,,26.8687,30.8477,,,12.366,6.62737,24.3831,,7.29621


In [ ]:
ICT['IC08Q02']['Perc'].head(2)

In [19]:
# All .xls files from C_comp_PAQ_MAR31

PAQ = defaultdict()
sheets = ['Perc', 'Mean_C_PROB', 'Mean_C_MATH', 'Mean_C_READ']

base_dir = "/Users/nichollejames/Desktop/capstone_project/C_comp_PAQ_MAR31/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        for sheet in sheets:
            
            key_failed = []
            try:
                update_dict = {
                    sheet:  pd.read_excel(base_dir+f, sheetname=sheet)       
                }
            
                if key not in PAQ:
                    PAQ[key] = {}
            
                PAQ[key].update(update_dict)

# ICT[x][y] change all of the column names

            for x in PAQ:
                for y in sheets:
                    PAQ[x][y] = PAQ[x][y].rename(columns={' ': 'cnt',
                                                        'N': 'total_number_of_resp',
                                                        'N/A': 'pct_not_valid',
                                                        'N.1': 'no_of_valid_resp',
                                                        'No time': 'resp_no_time_pct',
                                                        'No time.1': 'resp_no_time_se',
                                                        'No time  ': 'resp_no_time_mean',
                                                        'Yes, students did this': 'resp_yes_students_did_this_pct',
                                                        'Yes, students did this.1': 'resp_yes_students did_this_time_se',
                                                        'Yes, but only the teacher demonstrated this': 'resp_yes_but_only_the_teacher_demonstrated_this_pct',
                                                        'Yes, but only the teacher demonstrated this.1': 'resp_yes_but_only_the_teacher_demonstrated_this_se',
                                                        '1-30 minutes': 'resp_1_30_minutes_pct',
                                                        '1-30 minutes.1': 'resp_1_30_minutes_se',
                                                        '31-60 minutes': 'resp_31_60_minutes_pct',
                                                        '31-60 minutes.1': 'resp_31_60_minute_se',
                                                        'Between 1 and 2 hours': 'resp_between_1_and_2_hours_pct',
                                                        'Between 1 and 2 hours.1': 'resp_between_1_and_2_hours_se',
                                                        'Between 2 and 4 hours': 'resp_between_2_and_4_hours_pct',
                                                        'Between 2 and 4 hours.1': 'resp_between_2_and_4_hours_se',
                                                        'Between 4 hours and 6 hours': 'resp_between_4_hours_and_6_hours_pct',
                                                        'Between 4 hours and 6 hours.1': 'resp_between_4_hours_and_6_hours_se',
                                                        'More than 6 hours': 'resp_more_than_6_hours_pct',
                                                        'More than 6 hours.1': 'resp_more_than_6_hours_se',
                                                        'Never or hardly ever': 'resp_never_or_hardly_ever_pct',
                                                        'Never or hardly ever.1': 'resp_never_or_hardly_ever_se',
                                                        'Once or twice a month' : 'resp_once_or_twice_a_month_pct',
                                                        'Once or twice a month.1': 'resp_once_or_twice_a_month_se',
                                                        'Once or twice a week' : 'resp_once_or_twice_a_week_pct',
                                                        'Once or twice a week.1': 'resp_once_or_twice_a_week_se',
                                                        'Almost every day': 'resp_almost_every_day_pct',
                                                        'Almost every day.1': 'resp_almost_every_day_se',
                                                        'Yes, and I use it': 'resp_yes_and_i_use_it_pct',
                                                        'Yes, and I use it.1': 'resp_yes_and_i_use_it_se',
                                                        'Yes, but I don?t use it': 'resp_yes_but_i_dont_use_it_pct',
                                                        'Yes, but I don?t use it.1': 'resp_yes_but_i_dont_use_it_se',
                                                        'No': 'resp_no_pct',
                                                        'No.1': 'resp_no_se',
                                                        'Strongly agree': 'resp_strongly_agree_pct',
                                                        'Strongly agree.1': 'resp_strongly_agree_se',
                                                        'Agree': 'resp_agree_pct',
                                                        'Agree.1': 'resp_agree_se',
                                                        'Disagree': 'resp_disagree_pct',
                                                        'Disagree.1': 'resp_disagree_se',
                                                        'Strongly disagree': 'resp_strongly_disagree_pct',
                                                        'Strongly disagree.1': 'resp_strongly_disagree_se',
                                                        'Every day': 'resp_every_day_pct',
                                                        'Every day.1': 'resp_every_day_se',
                                                        '6 years old or younger': 'resp_6_years_old_or_younger_pct',
                                                        '6 years old or younger.1': 'resp_6_years_old_or_younger_se',
                                                        '7-9 years old': 'resp_7_9_years_old_pct',
                                                        '7-9 years old.1': 'resp_7_9_years_old_se',
                                                        '10-12 years old': 'resp_10_12_years_old_pct',
                                                        '10-12 years old.1': 'resp_10_12_years_old_se',
                                                        '13 years old or older': 'resp_13_years_old_or_older_pct',
                                                        '13 years old or older.1': 'resp_13_years_old_or_older_se',
                                                        '13 years old  or older': 'resp_13_years_old_or_older_pct',
                                                        '13 years old  or older.1': 'resp_13_years_old_or_older_se',
                                                        'Never': 'resp_never_pct',
                                                        'Never.1': 'resp_never_se',
                                                        'Missing': 'pct__resp_missing',
                                                        'Missing.1': 'resp_missing_se'
                                 })
            except:
                key_failed.append(key)
            if key_failed:
                PAQ[key] = pd.read_excel(base_dir+f)


SyntaxError: invalid syntax (<ipython-input-19-ebfe564d53b7>, line 24)

In [17]:
# ICT[x][y] change all of the column names

for x in PAQ:
    for y in sheets:
        PAQ[x][y] = PAQ[x][y].rename(columns={' ': 'cnt',
                                            'N': 'total_number_of_resp',
                                            'N/A': 'pct_not_valid',
                                            'N.1': 'no_of_valid_resp',
                                            'No time': 'resp_no_time_pct',
                                            'No time.1': 'resp_no_time_se',
                                            'No time  ': 'resp_no_time_mean',
                                            'Yes, students did this': 'resp_yes_students_did_this_pct',
                                            'Yes, students did this.1': 'resp_yes_students did_this_time_se',
                                            'Yes, but only the teacher demonstrated this': 'resp_yes_but_only_the_teacher_demonstrated_this_pct',
                                            'Yes, but only the teacher demonstrated this.1': 'resp_yes_but_only_the_teacher_demonstrated_this_se',
                                            '1-30 minutes': 'resp_1_30_minutes_pct',
                                            '1-30 minutes.1': 'resp_1_30_minutes_se',
                                            '31-60 minutes': 'resp_31_60_minutes_pct',
                                            '31-60 minutes.1': 'resp_31_60_minute_se',
                                            'Between 1 and 2 hours': 'resp_between_1_and_2_hours_pct',
                                            'Between 1 and 2 hours.1': 'resp_between_1_and_2_hours_se',
                                            'Between 2 and 4 hours': 'resp_between_2_and_4_hours_pct',
                                            'Between 2 and 4 hours.1': 'resp_between_2_and_4_hours_se',
                                            'Between 4 hours and 6 hours': 'resp_between_4_hours_and_6_hours_pct',
                                            'Between 4 hours and 6 hours.1': 'resp_between_4_hours_and_6_hours_se',
                                            'More than 6 hours': 'resp_more_than_6_hours_pct',
                                            'More than 6 hours.1': 'resp_more_than_6_hours_se',
                                            'Never or hardly ever': 'resp_never_or_hardly_ever_pct',
                                            'Never or hardly ever.1': 'resp_never_or_hardly_ever_se',
                                            'Once or twice a month' : 'resp_once_or_twice_a_month_pct',
                                            'Once or twice a month.1': 'resp_once_or_twice_a_month_se',
                                            'Once or twice a week' : 'resp_once_or_twice_a_week_pct',
                                            'Once or twice a week.1': 'resp_once_or_twice_a_week_se',
                                            'Almost every day': 'resp_almost_every_day_pct',
                                            'Almost every day.1': 'resp_almost_every_day_se',
                                            'Yes, and I use it': 'resp_yes_and_i_use_it_pct',
                                            'Yes, and I use it.1': 'resp_yes_and_i_use_it_se',
                                            'Yes, but I don?t use it': 'resp_yes_but_i_dont_use_it_pct',
                                            'Yes, but I don?t use it.1': 'resp_yes_but_i_dont_use_it_se',
                                            'No': 'resp_no_pct',
                                            'No.1': 'resp_no_se',
                                            'Strongly agree': 'resp_strongly_agree_pct',
                                            'Strongly agree.1': 'resp_strongly_agree_se',
                                            'Agree': 'resp_agree_pct',
                                            'Agree.1': 'resp_agree_se',
                                            'Disagree': 'resp_disagree_pct',
                                            'Disagree.1': 'resp_disagree_se',
                                            'Strongly disagree': 'resp_strongly_disagree_pct',
                                            'Strongly disagree.1': 'resp_strongly_disagree_se',
                                            'Every day': 'resp_every_day_pct',
                                            'Every day.1': 'resp_every_day_se',
                                            '6 years old or younger': 'resp_6_years_old_or_younger_pct',
                                            '6 years old or younger.1': 'resp_6_years_old_or_younger_se',
                                            '7-9 years old': 'resp_7_9_years_old_pct',
                                            '7-9 years old.1': 'resp_7_9_years_old_se',
                                            '10-12 years old': 'resp_10_12_years_old_pct',
                                            '10-12 years old.1': 'resp_10_12_years_old_se',
                                            '13 years old or older': 'resp_13_years_old_or_older_pct',
                                            '13 years old or older.1': 'resp_13_years_old_or_older_se',
                                            '13 years old  or older': 'resp_13_years_old_or_older_pct',
                                            '13 years old  or older.1': 'resp_13_years_old_or_older_se',
                                            'Never': 'resp_never_pct',
                                            'Never.1': 'resp_never_se',
                                            'Missing': 'pct__resp_missing',
                                            'Missing.1': 'resp_missing_se'
                     })

KeyError: 'Perc'

In [ ]:
# Run this only once or the column names will have the keys repeated.

lvl1keys = ICT.keys()

for l1key in lvl1keys:
    lvl2keys = ICT[l1key].keys()
#     print lvl2keys
    for l2key in lvl2keys:
        current_columns = ICT[l1key][l2key].columns
        ICT[l1key][l2key].columns = [l1key+'_'+l2key+'_'+x if x!='cnt' else x for x in current_columns]
        

df_ICT = pd.DataFrame(ICT['IC01Q01']['Perc'])

for filename, worksheets in ICT.items():
    print "Merging: ", filename
    
    for worksheet, sheet_df in worksheets.items():
        # print worksheet, sheet_df.columns
        df_ICT = pd.merge(df_ICT, sheet_df, how="inner", on="cnt")

In [13]:
# All .xls files from C_comp_SCQ_MAR31

SCQ = defaultdict()
sheets = ['Perc', 'Mean_C_PROB', 'Mean_C_MATH', 'Mean_C_READ']

base_dir = "/Users/nichollejames/Desktop/capstone_project/C_comp_SCQ_MAR31/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        for sheet in sheets:
            
            key_failed = []
            try:
                update_dict = {
                    sheet:  pd.read_excel(base_dir+f, sheetname=sheet)       
                }
            
                if key not in SCQ:
                    SCQ[key] = {}
            
                SCQ[key].update(update_dict)
            except:
                key_failed.append(key)
            if key_failed:
                SCQ[key] = pd.read_excel(base_dir+f)
print key

SC52Q02


In [14]:
SCQ[SC52Q02]

NameError: name 'SC52Q02' is not defined

In [ ]:
# All xls files from C_comp_STU_MAR31
STU = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/C_comp_STU_MAR31/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        for sheet in sheets:
            
            key_failed = []
            try:
                update_dict = {
                    sheet:  pd.read_excel(base_dir+f, sheetname=sheet)       
                }
            
                if key not in SCQ:
                    SCQ[key] = {}
            
                SCQ[key].update(update_dict)
            except:
                key_failed.append(key)
            if key_failed:
                SCQ[key] = pd.read_excel(base_dir+f)

In [ ]:
# All xls files from PISA_results_2012/computer_based_assessment_2012/
CBA_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/computer_based_assessment_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        CBA_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# All xls files from PISA_results_2012/financial_lit_2012/
xls_to_pandas_FL_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/financial_lit_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        xls_to_pandas_FL_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# All xls files from PISA_results_2012/math_results_2012/
xls_to_pandas_MR_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/math_results_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        xls_to_pandas_MR_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# All xls files from PISA_results_2012/problem_solving_2012/
xls_to_pandas_PS_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/problem_solving_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        xls_to_pandas_PS_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# All xls files from PISA_results_2012/reading_lit_2012/
xls_to_pandas_RL_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/reading_lit_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        xls_to_pandas_RL_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# All xls files from PISA_results_2012/results_for_states_2012/
xls_to_pandas_RFS_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/results_for_states_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        xls_to_pandas_RFS_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# All xls files from PISA_results_2012/science_lit_2012/
xls_to_pandas_SL_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/science_lit_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        xls_to_pandas_SL_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# All xls files from PISA_results_2012/trends_2012/
xls_to_pandas_trends_2012 = {}

base_dir = "/Users/nichollejames/Desktop/capstone_project/PISA_results_2012/trends_2012/"
for f in os.listdir(base_dir): 
    if f.endswith(".xls"):
        key = f.split('.')[0]
        xls_to_pandas_trends_2012[key] = pd.read_excel(base_dir+f)


In [ ]:
# my code that didn't work
#     if sheetname == 'Perc':
#             EC[key] = pd.read_excel(base_dir+f, sheetname='Perc', skiprows = 1)
#         elif sheetname == 'Mean_C_PROB':
#             EC[key] = pd.read_excel(base_dir+f, sheetname='Mean_C_PROB', skiprows = 1)
#         elif sheetname == 'Mean_C_MATH':
#             EC[key] = pd.read_excel(base_dir+f, sheetname='Mean_C_MATH', skiprows = 1)
#         elif sheetname == 'Mean_C_READ':
#             EC[key] = pd.read_excel(base_dir+f, sheetname='Mean_C_READ', skiprows = 1)
#         else:
#             EC[key] = pd.read_excel(base_dir+f, skiprows = 1)